# Práctica de Laboratorio N°3: Recepción de hipótesis enviadas a través del canal inhalámbrico.

## Objetivo:
Analizar la transmisión y recepción de señales digitales complejas binarias usando un SDR:
- Observar las características temporales, frecuenciales y estadísticas de una señal binaria compleja.
- Ver cómo se afectan estas características al modificar ciertos parámetros como:

    ref: que ajusta la probabilidad de ocurrencia de los valores binarios de la señal.
    
    TxAtten: que ajusta la potencia de transmisión del SDR.
    
- Enviar señales con una distribución diferente (no binaria), y observar su comportamiento luego de la transmisión.

# Señal binaria a enviar
El siguiente código genera una señal (hipótesis en este problema) binaria comleja que permite modificar, a través de la variable "ref", la probabilidad de ocurrencia de los valores binarios. En prencipio vamos a suponer que los mensajes son equiprobables (ref = 0.5):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq

ref = 0.5
nb_samples = 100

# Generate random bits
z = np.random.uniform(size=nb_samples) # Generate random numbers between 0 and 1
# Convert to binary signal based on reference value 
x = [1 if z[i]>ref else 0 for i in range(len(z))] 
txSignal = (2*np.repeat(x, 2**4)-1)+1j*(2*np.repeat(x, 2**4)-1) # Transmit signal

Se genrea un array de numeros aleatorio z que contiene numeros distribuidos uniformemente entre 0 y 1. Luego, a partir del valor ref, se convierte cada elemento de z en un valor binario: si z[i] > ref, el valor es 1, si no, es 0. Cada valor binario se almacena en el array x. 

Al momento de la transmisión, se repite cada valor de x un total de 16 veces, generando una señal más larga manteniendo cada valor constante por un período. Ademas, se transforma cada 0 y 1 en -1 y +1 respectivamente, construyendo asi una señal compleja binaria.

### Gráfica temporal

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(np.real(txSignal), label='Parte real')
plt.plot(np.imag(txSignal), label='Parte imaginaria', alpha=0.7)
plt.title('Señal en el tiempo')
plt.xlabel('Muestra')
plt.ylabel('Amplitud')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

En la gráfica temporal se observa que tanto la componente en fase (parte real) como la componente en cuadratura (parte imaginaria) presentan niveles constantes en -1 y +1, alternando aleatoriamente. Cada valor se mantiene constante durante 16 muestras consecutivas, ya que se utilizó un factor de repetición (2^4 = 16) para extender temporalmente cada símbolo.

### Histogramas

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(np.real(txSignal), bins=np.arange(-1.5, 2, 1), edgecolor='black')
plt.title('Histograma Parte Real')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')

plt.subplot(1, 2, 2)
plt.hist(np.imag(txSignal), bins=np.arange(-1.5, 2, 1), edgecolor='black')
plt.title('Histograma Parte Imaginaria')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')

plt.tight_layout()
plt.show()

Los histogramas de las componentes real e imaginaria muestran claramente dos picos centrados en los valores -1 y +1. Como la probabilidad de 0 y 1 fue de 50% (por ref = 0.5), ambos valores aparecen con aproximadamente la misma frecuencia, lo que indica una distribución balanceada. Esta distribución simétrica es esperable en una señal binaria generada con igual probabilidad para ambos símbolos.

### Gráfica de frecuencias

In [ ]:
# Transformada de Fourier
N = len(txSignal)
frecuencia = fftfreq(N, d=1)  # Frecuencias
espectro = fft(txSignal)

plt.figure(figsize=(12, 4))
plt.plot(np.abs(frecuencia[:N // 2]), 20*np.log10(np.abs(espectro[:N // 2])))
plt.title('Magnitud del espectro de la señal')
plt.xlabel('Frecuencia (Hz normalizada)')
plt.ylabel('Magnitud (dB)')
plt.grid(True)
plt.tight_layout()
plt.show()

El espectro de la señal presenta una estructura periódica con caídas pronunciadas (notches) en intervalos regulares. Esto se debe a que cada símbolo binario (0 o 1) fue repetido 16 veces consecutivas (2^4), generando un patrón determinista en el tiempo. Esta repetición introduce correlación entre muestras, lo que se traduce en el dominio de la frecuencia en una envolvente periódica con mínimos regulares.

# Configuración del SDR
Se utilizan las siguientes librerias para conectarnos al SDR y configurar los parametros:

In [ ]:
#!pip install pyadi-iio
#!pip install seaborn

La configuración para conectarnos al SDR es la siguiente:

In [ ]:
import adi

# Configuración del SDR
sdr = adi.Pluto("ip:192.168.1.32")
# Parámetros TX y RX
sdr.sample_rate = int(2e6)              # Tasa de muestreo
sdr.tx_lo = int(915e6)                  # Frecuencia de transmisión (ejemplo)
sdr.tx_hardwaregain = -79               # Atenuación en TX [dB]
sdr.tx_rf_bandwidth = int(2e6)          # Ancho de banda TX

sdr.rx_lo = sdr.tx_lo                   # Frecuencia de recepción igual a la de TX
sdr.rx_gain_control_mode = "manual"
sdr.rx_hardwaregain = 78
sdr.rx_rf_bandwidth = sdr.tx_rf_bandwidth
sdr.rx_buffer_size = len(txSignal)

# Transmisión continua
sdr.tx_cyclic_buffer = True

## Transmisión y recepción de la señal usando el SDR

In [ ]:
# Cargar y transmitir la señal
sdr.tx(txSignal * (2**14))  # Transmisión
print("Transmisión iniciada...")

# Limpieza del buffer de recepción  
for i in range(0, 10):
    raw_data = sdr.rx()

# Recepción
rxSignal = sdr.rx()
print("Recepción completa.")

# Normalización a potencia unitaria
rxSignal /= np.sqrt(np.mean(np.abs(rxSignal)**2))

# Cierre del dispositivo
sdr.tx_destroy_buffer()  # Libera el buffer de transmisión

Se realiza la emisión y recepción de la señal con el SDR. Se tiene en cuenta:
- Limpieza del buffer de recepcion: Se descartan las primeras 10 lecturas del receptor ya que el SDR puede tardar unos ciclos en estabilizarse.
- Normalización de la potencia de la señal: Para asegurarnos de que la potencia de la señal esté dentro de un rango estandar, se la normaliza. Se calcula el modulo de cada muestra compleja (amplitud) para luego calcular la potencia promedio de la señal. Calculando la raiz cuadrada de la potencia promedio obtengo el valor RMS de la potencia. Dividiendo la señal receptada por su potencia en RMS, se obtiene la señal con potencia unitaria.

### Gráfica de los resultados

In [ ]:
# Señales I y Q separadas
plt.figure(figsize=(10, 4))
plt.subplot(2, 1, 1)
plt.plot(np.real(rxSignal), label="En fase (I)", color='blue')
plt.title("Componente en fase (I)")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(np.imag(rxSignal), label="En cuadratura (Q)", color='red')
plt.title("Componente en cuadratura (Q)")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Histogramas
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(np.real(rxSignal), bins=50, color='blue', alpha=0.7)
plt.title("Histograma de componente I")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.hist(np.imag(rxSignal), bins=50, color='red', alpha=0.7)
plt.title("Histograma de componente Q")
plt.grid(True)

plt.tight_layout()
plt.show()

Respecto a la señal recibida en el dominio temporal, se observa que en una seccion inicial encontramos mucho ruido, probablemente en el preambulo o parte inestable de la transmisión. Luego, entre las muestras 4000 y 16000 la señal parece estable y con estructura repetitiva, lo cual indica que la transmisión fue captada correctamente.

En los histogramas de las componenetes I y Q, se observan 2 picos en cada histograma, sugiriendo que se está recibiendo una modulación en cuadratura con 2 niveles, como QPSK, donde cada componente toma valores timicamente cercanos a {-1, +1}.

## Transmisión con mayor potencia
Repetimos los pasos anteriores pero aumentando la potencia de la señal transmitida, llevando el valor de la variable "TxAtten" al valor -70. 

In [ ]:
sdr.tx_hardwaregain = -70               # Atenuación en TX [dB]

sdr.tx(txSignal * (2**14))

for i in range(0, 10):
    raw_data = sdr.rx()

rxSignal2 = sdr.rx()
rxSignal2 /= np.sqrt(np.mean(np.abs(rxSignal2)**2))
sdr.tx_destroy_buffer()

plt.figure(figsize=(10, 4))
plt.subplot(2, 1, 1)
plt.plot(np.real(rxSignal), label="En fase (I)", color='blue')
plt.title("Componente en fase (I)")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(np.imag(rxSignal), label="En cuadratura (Q)", color='red')
plt.title("Componente en cuadratura (Q)")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(np.real(rxSignal2), bins=50, color='blue', alpha=0.7)
plt.title("Histograma de componente I")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.hist(np.imag(rxSignal2), bins=50, color='red', alpha=0.7)
plt.title("Histograma de componente Q")
plt.grid(True)

plt.tight_layout()
plt.show()

Analizando el grafico en el dominio temporal:
- Antes, la señal recibida tenia menor amplitud, con componentes I y Q más comprimidos.
- Ahora, se observa un leve aumento en la amplitud de la señal recibida. Esto es esperable ya que a menor atenuación, mayor potencia transmitida y mayor señal recibida.

Por otro lado, analizando los histogramas, vemos que los picos se mantiene cercanos a los valores -1 y +1, sin modificarse demasiado su distribución al comparar las dos transmisiones.

## Transmisión con probabilidades a priori distintas

Configuramos el valor de la variable red a un valor 0.3 y repitiendo los pasos anteriores analizamos y comparamos los resultados obtenidos:

In [ ]:
ref = 0.3

z = np.random.uniform(size=nb_samples)
x = [1 if z[i]>ref else 0 for i in range(len(z))] 
txSignal3 = (2*np.repeat(x, 2**4)-1)+1j*(2*np.repeat(x, 2**4)-1) 

sdr.tx_hardwaregain = -79

sdr.tx(txSignal3 * (2**14))

for i in range(0, 10):
    raw_data = sdr.rx()

rxSignal3 = sdr.rx()
rxSignal3 /= np.sqrt(np.mean(np.abs(rxSignal3)**2))
sdr.tx_destroy_buffer()

plt.figure(figsize=(10, 4))
plt.subplot(2, 1, 1)
plt.plot(np.real(rxSignal), label="En fase (I)", color='blue')
plt.title("Componente en fase (I)")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(np.imag(rxSignal), label="En cuadratura (Q)", color='red')
plt.title("Componente en cuadratura (Q)")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(np.real(rxSignal3), bins=50, color='blue', alpha=0.7)
plt.title("Histograma de componente I")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.hist(np.imag(rxSignal3), bins=50, color='red', alpha=0.7)
plt.title("Histograma de componente Q")
plt.grid(True)

plt.tight_layout()
plt.show()

La señal recibida en el dominio temporal manteine las amplitudes similares a transmisiones anteriores. Visualmente, parece que hay mas repeticiones de ciertos niveles, lo que puede indicar que uno de los simbolos se está transmitiendo con mayor frecuencia. 

Analizando loas histogramas, antes teniamos dos picos aproximadamente simetricos. Ahora, los picos entorno a -1 y +1 tienen amplitudes desiguales. Esto significa que los valores de las componentes tienden a concentrarse más en una region, lo cual indica que se transmite mas frecuentemente el simbolo correspondiente a ese nivel de amplitud.

Cuando la probabilidad de transmision de cada bit es distinta, los histogramas resultan asimetricos y tendremos menor entropía en la fuente de información ya que un simbolo domina.

# Transmision de señal no binaria

Definimos ahora la señal de manera que transmita una hipotesis H que pertenexca al conjunto {-1, -0.3333, +0.3333, +1} con igual probabilidades. Se repiten los pasos de transmision y recepcion para comprar los resultados con las transmisiones anteriores.

In [ ]:
x = np.array([-1, -0.3333, 0.3333, 1])
nb_samples = 10

I = np.random.choice(x, nb_samples, p=[0.25, 0.25, 0.25, 0.25])  # Generar parte real aleatoria con igual probabilidad
Q = np.random.choice(x, nb_samples, p=[0.25, 0.25, 0.25, 0.25])  # Generar parte imaginaria aleatoria con igual probabilidad

txSignal4 = I + 1j*Q  # Señal transmitida

sdr.tx(txSignal4 * (2**14))

for i in range(0, 10):
    raw_data = sdr.rx()

rxSignal4 = sdr.rx()
rxSignal4 /= np.sqrt(np.mean(np.abs(rxSignal4)**2))
sdr.tx_destroy_buffer()

plt.figure(figsize=(10, 4))
plt.subplot(2, 1, 1)
plt.plot(np.real(rxSignal), label="En fase (I)", color='blue')
plt.title("Componente en fase (I)")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(np.imag(rxSignal), label="En cuadratura (Q)", color='red')
plt.title("Componente en cuadratura (Q)")
plt.xlabel("Muestras")
plt.ylabel("Amplitud")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.hist(np.real(rxSignal4), bins=50, color='blue', alpha=0.7)
plt.title("Histograma de componente I")
plt.grid(True)

plt.subplot(1, 2, 2)
plt.hist(np.imag(rxSignal4), bins=50, color='red', alpha=0.7)
plt.title("Histograma de componente Q")
plt.grid(True)

plt.tight_layout()
plt.show()


En las graficas temporales de la señal recibida es posible ver que ahora la señal no llega a valer 0 si no un valor cercano (-0.3333 y +0.3333) en cada cambio de valor trasnmitido. Hay presencia de ruido pero aun puede distinguirse claramente los niveles.

Respecto a los histogramas, ahora no tenemos dos picos pronunciados en -1 y +1, sino cuatro ubicados aproximadamente en -1, -0.3333, 0.3333 y 1. La altura de los picos es similar, indicando que los simbolos tienen igual probabilidad. La forma de cada pico sigue una distribucion tipo gaussiana debido al ruido aditivo.